In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from fastai.vision.all import *

In [ ]:
train_df=pd.read_csv("../input/microsoft-rice-disease-classification-challenge/Train (6).csv")
test_df=pd.read_csv("../input/microsoft-rice-disease-classification-challenge/Test (8).csv")
sub=pd.read_csv("../input/microsoft-rice-disease-classification-challenge/SampleSubmission (4).csv")

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
Id=test_df["Image_id"]

In [ ]:
train_df.Label.unique()

In [ ]:
train_df.Label.value_counts()

In [ ]:
# dls = ImageDataLoaders.from_df(train_df,bs=32, path='../input/microsoft-rice-disease-classification-challenge/Images (1)/', num_workers=-1) # See the docs for adding augmentations etc
# dls.show_batch() 


In [ ]:
path='../input/microsoft-rice-disease-classification-challenge/Images (1)/'
img_paths = []
for i in train_df.Image_id:
    img_paths.append(path + i)
train_df['path'] = img_paths

In [ ]:
train_df

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()

train_df= pd.get_dummies(train_df, columns = ["Label"])

In [ ]:
train_df

In [ ]:
train_datagenerator = ImageDataGenerator(
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    rescale=1./255, 
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.1)

In [ ]:
train_generator = train_datagenerator.flow_from_dataframe(
    train_df, 
    x_col='path', 
    y_col=['Label_blast',"Label_brown","Label_healthy"],
    target_size=(120, 120), 
    color_mode='rgb', 
    class_mode='raw', 
    batch_size=32, 
    shuffle=True, 
    seed=1,
    subset='training')
validation_generator = train_datagenerator.flow_from_dataframe(
    train_df , 
    x_col='path',
    y_col=['Label_blast',"Label_brown","Label_healthy"],
    target_size=(120, 120),
    color_mode='rgb', 
    class_mode='raw', 
    batch_size=4, 
    shuffle=True,
    seed=1, 
    subset='validation')

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 6):
    plt.subplot(2, 3, i+1)
    for X_batch, Y_batch in validation_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
train_df=train_df.drop("Image_id", axis=1, inplace=True)

In [ ]:
model = ks.models.Sequential()

model.add(ks.layers.Dense(4, input_shape=(120, 120, 3)))

model.add(ks.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(ks.layers.MaxPooling2D(2, 2))

model.add(ks.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(ks.layers.MaxPooling2D(2, 2))


model.add(ks.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(ks.layers.MaxPooling2D(2, 2))

model.add(ks.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(ks.layers.MaxPooling2D(2, 2))
model.add(ks.layers.Dropout(0.2))

        
model.add(ks.layers.Flatten())
        
model.add(ks.layers.Dense(1024, activation='relu'))

model.add(ks.layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

Use focal loss

In [ ]:
batch_size=15
FAST_RUN = False
epochs=5 if FAST_RUN else 100
history = model.fit_generator(
    train_generator, 
    epochs=1,
    validation_data=validation_generator
)

In [ ]:
model.summary()

In [ ]:
test_df

In [ ]:
from PIL import Image

In [ ]:
for i in range(test_df.shape[0]//2):
    index=i*2
    img_path = test_df.loc[index,"Image_id"]
    path = "../input/microsoft-rice-disease-classification-challenge/Images (1)/" + img_path
    img = np.array(Image.open(path).resize((120,120))).reshape((1, 120, 120, 3))
    out = model(img)
    print(tf.argmax(out, axis=1).item())

In [ ]:
sub